In [13]:
import notebook_common
import jaconv
import cutlet
import os
from importlib import reload

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    # read_excel,
    to_csv_sig,
    # ------------------2400 word------------------
    # r_word_2400_xlsx,
    # w_word_2400_csv,
    # w_word_2400_with_guide_csv,
    w_word_2400_without_guide_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    # pattern_table_written,
    # pattern_td_written,
    # r_written_freq_nlt_with_enter_txt,
    # w_written_freq_nlt_with_enter_csv,
    w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    # pattern_table_spoken,
    # pattern_td_spoken,
    # r_spoken_freq_nlb_with_enter_txt,
    # w_spoken_freq_nlb_with_enter_csv,
    # w_spoken_freq_nlb_with_enter_filtered_csv,
    # w_spoken_freq_nlb_with_enter_filtered_hrt_csv,
    w_spoken_freq_nlb_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    w_merged_freq_sumed_csv,
    w_raw_merged_freq_sumed_csv,
    w_word_raw_merged_freq_sumed_csv,
    # ------------------公共python库------------------
    pd,
    np,
    p,
    l,
)

p("Import common module")

Import common module


In [14]:
df_spoken = pd.read_csv(
    w_spoken_freq_nlb_sumed_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    index_col=["headword", "hira"],
)

df_written = pd.read_csv(
    w_written_freq_nlt_sumed_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    index_col=["headword", "hira"],
)


df_w_word_2400_without_guide_csv = pd.read_csv(
    w_word_2400_without_guide_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    # index_col=["word"],
)
l("df_spoken")
df_spoken

----------------------------------------df_spoken----------------------------------------


,,spoken_freq
headword,hira,
の,の,57221425
に,に,35286930
を,を,33707106
は,は,31279365
が,が,30705135
...,...,...
ｋワット,きろわっと,1
右顧,うこ,1
右脳する,うのうする,1


In [15]:
l("df_w_word_2400_without_guide_csv")
df_w_word_2400_without_guide_csv

----------------------------------------df_w_word_2400_without_guide_csv----------------------------------------


,number,word,hira,pos,prop1,sub_prop1,prop2,sub_prop2,prop3,sub_prop3,remark,meaning,cnt-word,cnt-word-hira
0,1,ああ,ああ,NaN,感叹词,NaN,NaN,NaN,NaN,NaN,NaN,啊、嗯、啊呀、唉,1,1
1,2,愛,あい,五十音,名词,NaN,动词,动3,NaN,NaN,他,爱情、爱好,1,1
2,3,あいさつ,あいさつ,NaN,名词,NaN,动词,动3,NaN,NaN,自,寒暄、致辞,1,1
3,4,合図,あいず,NaN,名词,NaN,NaN,NaN,NaN,NaN,NaN,信号,1,1
4,5,アイスクリーム,あいすくりーむ,NaN,名词,NaN,NaN,NaN,NaN,NaN,NaN,冰激凌、雪糕,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,象,ぞう,Ｌ13-2,名词,NaN,NaN,NaN,NaN,NaN,NaN,大象,1,1
2460,2461,東京,とうきょう,Ｌ13-2,名词,NaN,NaN,NaN,NaN,NaN,NaN,东京,1,1
2461,2462,居間,いま,Ｌ14,名词,NaN,NaN,NaN,NaN,NaN,NaN,起居室，起坐间,1,1
2462,2463,中学生,ちゅうがくせい,Ｌ14,名词,NaN,NaN,NaN,NaN,NaN,NaN,中学生,1,1


In [16]:
# 输出口语和书面语交集，索引是联合索引 拼写-发音
ids_df_spoken = set(df_spoken.index)
p(f"spoken freq index len:{len(ids_df_spoken)}")
ids_df_written = set(df_written.index)
p(f"written freq index len:{len(ids_df_written)}")
common_ids = ids_df_spoken & ids_df_written
p(f"common index len:{len(common_ids)}")

spoken freq index len:96734
written freq index len:84530
common index len:77257


In [17]:
# 原始书面语和口语词频合并
df_merged_raw = pd.concat([df_spoken, df_written], axis=1, join="outer")

# 将2400条高考词汇，包含拼写和发音
df_w_word_2400_without_guide_csv_word_hira = df_w_word_2400_without_guide_csv[
    ["word", "hira"]
].drop_duplicates(subset=["word", "hira"])

df_merged_raw.reset_index(inplace=True)
df_merged_raw = pd.merge(
    df_merged_raw,
    df_w_word_2400_without_guide_csv_word_hira,
    left_on=["headword", "hira"],
    right_on=["word", "hira"],
    how="outer",
    indicator="check",
)

# 空值补0
df_merged_raw["spoken_freq"] = df_merged_raw["spoken_freq"].fillna(0)
df_merged_raw["written_freq"] = df_merged_raw["written_freq"].fillna(0)
df_merged_raw["headword"] = df_merged_raw["headword"].fillna(df_merged_raw["word"])
df_merged_raw["cnt_headword_hira"] = df_merged_raw.groupby(
    ["headword", "hira"]
).transform("size")
df_merged_raw["cnt_headword"] = df_merged_raw.groupby(["headword"]).transform("size")

# 设置word_type {1为高中2400词汇;  0为默认类型}
df_merged_raw["spell_pron_type"] = 0
df_merged_raw.loc[df_merged_raw["check"].isin(["both", "right_only"]), "spell_pron_type"] = 1

# 设置has_freq {1 有词频数据  0无词频数据}
df_merged_raw["has_freq"] = 1
df_merged_raw.loc[df_merged_raw["check"].isin(["right_only"]), "has_freq"] = 0

del df_merged_raw['word']
del df_merged_raw['check']

to_csv_sig(df_merged_raw, w_raw_merged_freq_sumed_csv)
l("df_merged_raw--ok")
df_merged_raw.head()

----------------------------------------df_merged_raw--ok----------------------------------------


,headword,hira,spoken_freq,roman,written_freq,cnt_headword_hira,cnt_headword,spell_pron_type,has_freq
0,110番,ひゃくとおばん,0.0,hyakutooban,108.0,1,1,0,1
1,110番する,ひゃくとおばんする,0.0,hyakutoobansuru,7.0,1,1,0,1
2,1つ,ひとつ,0.0,hitotsu,4366.0,1,1,0,1
3,1つする,ひとつする,0.0,hitotsusuru,2.0,1,1,0,1
4,2ラン,つーらん,0.0,tsuran,99.0,1,1,0,1


In [18]:
# 作标准化处理 ，统计计算每4294,0640的词频, 以书面语词频作为基准
sum_spoken_freq = df_merged_raw["spoken_freq"].sum()
sum_written_freq = df_merged_raw["written_freq"].sum()
p(f"sum_spoken_freq:{sum_spoken_freq}")
p(f"sum_written_freq:{sum_written_freq}")
base_spoken_rate = sum_spoken_freq / sum_written_freq
# base_written_rate = sum_written_freq/10000000
p(f"base_spoken_rate:{base_spoken_rate}")
# p(base_written_rate)

sum_spoken_freq:949781504.0
sum_written_freq:42940518.0
base_spoken_rate:22.11853857934364


In [19]:
# 关联mp3发音文件
if os.path.exists("D:/Dropbox/06.wanjuan/99.tmp/df_ff_5.csv"):
    df_ff_5 = pd.read_csv(
        "D:/Dropbox/06.wanjuan/99.tmp/df_ff_5.csv", encoding="utf-8-sig", on_bad_lines="skip", engine="python"
    )
    l("ok")
else:
    df_ff_4 = pd.read_csv(
        "D:/04.japandict/Forvo/ff-4.csv",
        encoding="utf-8-sig",
        on_bad_lines="skip",
        engine="python",
    )
    df_ff_4.dropna(subset=["headword_voice", "mp3_path"], inplace=True)

    df_ff_4["headword_voice"] = df_ff_4["headword_voice"].apply(jaconv.kata2hira)

    df_ff_5 = (
        df_ff_4.groupby("headword_voice")
        .agg({"mp3_path": lambda x: "|".join(x[x != ""])})  # 字符串拼接  # 求平均值
        .reset_index()
    )
    l("df_ff_5")
    to_csv_sig(df_ff_5, "D:/Dropbox/06.wanjuan/99.tmp/df_ff_5.csv")

----------------------------------------ok----------------------------------------


In [20]:
# 对词频做标准化处理 ，对书面词词频增大22倍，之后和口语词频作加权

# 加权系数
spoken_weight, written_weight = 0.667, 0.333

# 融合词频
df_merged_raw["fused_freq"] = round(
    df_merged_raw["spoken_freq"] * spoken_weight
    + df_merged_raw["written_freq"] * written_weight * base_spoken_rate
)

# df_merged_raw["freq_for_order"]=df_merged_raw['fused_freq']

# 计算发音的词频占比
df_merged_raw["freq_sum_headword"] = df_merged_raw.groupby("headword")[
    "fused_freq"
].transform("sum")
df_merged_raw["hira_freq_per"] = round(
    df_merged_raw["fused_freq"] / df_merged_raw["freq_sum_headword"], 2
)


# 生成平假名发音id，2位数字，从10开始
df_merged_raw = df_merged_raw.sort_values(
    ["headword", "fused_freq"], ascending=[True, False]
)

df_merged_raw["hira_order"] = df_merged_raw.groupby("headword").cumcount() + 10
df_merged_raw["hira_order"] = df_merged_raw["hira_order"].astype(str).str.zfill(2)
df_merged_raw["pron_index"] = df_merged_raw.groupby("headword").cumcount() + 1
# df_merged_raw.sort_values(by="fused_freq", ascending=False, inplace=True)
df_merged_raw.sort_values(
    ["freq_sum_headword", "headword", "hira_order"],
    ascending=[False, False, True],
    inplace=True,
)

to_csv_sig(df_merged_raw, "D:/Dropbox/06.wanjuan/99.tmp/hira_order.csv", True)
l("ok")

----------------------------------------ok----------------------------------------


In [21]:
df_merged_raw

,headword,hira,spoken_freq,roman,written_freq,cnt_headword_hira,cnt_headword,spell_pron_type,has_freq,fused_freq,freq_sum_headword,hira_freq_per,hira_order,pron_index
6733,の,の,57221425.0,no,686025.0,1,1,0,1,43219589.0,43219589.0,1.0,10,1
6391,に,に,35286930.0,NaN,0.0,1,1,0,1,23536382.0,23536382.0,1.0,10,1
9465,を,を,33707106.0,NaN,0.0,1,1,0,1,22482640.0,22482640.0,1.0,10,1
6865,は,は,31279365.0,NaN,0.0,1,1,0,1,20863336.0,20863336.0,1.0,10,1
2400,が,が,30705135.0,NaN,0.0,1,1,0,1,20480325.0,20480325.0,1.0,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,いい／よい,いい／よい,0.0,NaN,0.0,1,1,1,0,0.0,0.0,NaN,10,1
387,あふれる,あふれる,0.0,NaN,0.0,1,1,1,0,0.0,0.0,NaN,10,1
378,あの人,あのひと,0.0,NaN,0.0,1,1,1,0,0.0,0.0,NaN,10,1
333,あちこち,あちこち,0.0,NaN,0.0,1,1,1,0,0.0,0.0,NaN,10,1


In [22]:
# 重设df_merged_raw的索引， 设为只有headword
df_merged_raw_word = df_merged_raw.set_index("headword")
df_merged_raw_word[0:3]

,hira,spoken_freq,roman,written_freq,cnt_headword_hira,cnt_headword,spell_pron_type,has_freq,fused_freq,freq_sum_headword,hira_freq_per,hira_order,pron_index
headword,,,,,,,,,,,,,
の,の,57221425.0,no,686025.0,1,1,0,1,43219589.0,43219589.0,1.0,10,1
に,に,35286930.0,NaN,0.0,1,1,0,1,23536382.0,23536382.0,1.0,10,1
を,を,33707106.0,NaN,0.0,1,1,0,1,22482640.0,22482640.0,1.0,10,1


In [23]:
# 词汇以headword分组，对词频汇总
df_merged_raw_word_agg = (
    df_merged_raw_word.groupby(df_merged_raw_word.index)
    .agg(
        {
            "fused_freq": lambda x: sum(x),
            "spoken_freq": lambda x: sum(x),
            "written_freq": lambda x: sum(x),
        }
    )
    .assign(pron_count=lambda x: df_merged_raw_word.index.value_counts())
    # 字符串拼接  # 求平均值
    .reset_index()
)
l("ok")
df_merged_raw_word_agg

----------------------------------------ok----------------------------------------


,headword,fused_freq,spoken_freq,written_freq,pron_count
0,110番,795.0,0.0,108.0,1
1,110番する,52.0,0.0,7.0,1
2,1つ,32158.0,0.0,4366.0,1
3,1つする,15.0,0.0,2.0,1
4,2ラン,729.0,0.0,99.0,1
...,...,...,...,...,...
102508,ｓｕｐｅｒ,2.0,3.0,0.0,1
102509,ｔ,1174.0,1760.0,0.0,1
102510,ｔｈｅ,40.0,60.0,0.0,1
102511,ｖ,79.0,119.0,0.0,1


In [24]:
# 输出以word为基础的词频表
# df_merged_raw_word_agg.sort_values(by="fused_freq", ascending=False, inplace=True)
# start_index = 100000
# df_merged_raw_word_agg.index = range(
#     start_index, start_index + len(df_merged_raw_word_agg)
# )
# df_merged_raw_word_agg["frequency"] = range(1, 1 + len(df_merged_raw_word_agg))

# ---------------------------------------------
# 把高中2400的词汇加入
# 先将原始10万词条和高考2400词条作outer join

df_w_word_2400_without_guide_csv_new = df_w_word_2400_without_guide_csv[
    ["word"]
].drop_duplicates(subset=["word"])

df_merged_raw_word_agg = pd.merge(
    df_merged_raw_word_agg,
    df_w_word_2400_without_guide_csv_new,
    left_on="headword",
    right_on="word",
    how="outer",
    indicator="check",
)

df_merged_raw_word_agg["spell"] = df_merged_raw_word_agg["headword"].fillna(
    df_merged_raw_word_agg["word"]
)
# 设置word_type 1为高中2400词汇， 0为默认类型
df_merged_raw_word_agg["spell_type"] = 0
df_merged_raw_word_agg.loc[
    df_merged_raw_word_agg["check"].isin(["both", "right_only"]), "spell_type"
] = 1

df_merged_raw_word_agg.sort_values(by="fused_freq", ascending=False, inplace=True)
start_index = 100000
df_merged_raw_word_agg.index = range(
    start_index, start_index + len(df_merged_raw_word_agg)
)
df_merged_raw_word_agg["frequency"] = range(1, 1 + len(df_merged_raw_word_agg))


# 1：已有词性词义例句数据       0：默认
# df_outer_joiner["has_data"] = 0
# df_outer_joiner.loc[df_outer_joiner["check"].isin(["both"]), "has_data"] = 1

# null_word_id_indices = df_outer_joiner["headword"].isnull()
# num_null_word_ids = null_word_id_indices.sum()

# if num_null_word_ids > 0:
#     start_id = 210000
#     new_ids = np.arange(start_id, start_id + num_null_word_ids)
#     df_outer_joiner.loc[null_word_id_indices, "word_id"] = new_ids


# -------------------------------------------------------
df_merged_raw_word_agg.index.name = "word_id"
df_merged_raw_word_agg_order = [
    "spell",
    "frequency",
    "spoken_freq",
    "written_freq",
    "fused_freq",
    "pron_count",
    "spell_type",
]
# to_csv_sig(df_outer_joiner, "D:/Dropbox/06.wanjuan/99.tmp/df_outer_joiner_2.csv",True)
df_merged_raw_word_agg = df_merged_raw_word_agg[df_merged_raw_word_agg_order]
df_merged_raw_word_agg = df_merged_raw_word_agg.rename(columns={"spell": "headword"})
to_csv_sig(df_merged_raw_word_agg, w_word_raw_merged_freq_sumed_csv, True)
l("df_merged_raw_word_agg--ok")
df_merged_raw_word_agg

----------------------------------------df_merged_raw_word_agg--ok----------------------------------------


,headword,frequency,spoken_freq,written_freq,fused_freq,pron_count,spell_type
word_id,,,,,,,
100000,の,1,57221425.0,686025.0,43219589.0,1,0
100001,に,2,35286930.0,0.0,23536382.0,1,0
100002,を,3,33707106.0,0.0,22482640.0,1,0
100003,は,4,31279365.0,0.0,20863336.0,1,0
100004,が,5,30705135.0,0.0,20480325.0,1,0
...,...,...,...,...,...,...,...
202508,友だち,102509,0.0,0.0,0.0,1,1
202509,範圍,102510,0.0,0.0,0.0,1,1
202510,勿論,102511,0.0,0.0,0.0,1,1


In [25]:
# df_merged_raw 关联 df_merged_raw_word_agg的ID主键
df_merged_raw_word_agg.reset_index(inplace=True)
df_merged_raw_word_agg = df_merged_raw_word_agg.rename(
    columns={"fused_freq": "word_fused_freq"}
)
df_merged_raw_word_agg.head()

,word_id,headword,frequency,spoken_freq,written_freq,word_fused_freq,pron_count,spell_type
0,100000,の,1,57221425.0,686025.0,43219589.0,1,0
1,100001,に,2,35286930.0,0.0,23536382.0,1,0
2,100002,を,3,33707106.0,0.0,22482640.0,1,0
3,100003,は,4,31279365.0,0.0,20863336.0,1,0
4,100004,が,5,30705135.0,0.0,20480325.0,1,0


In [26]:
df_merged_raw

,headword,hira,spoken_freq,roman,written_freq,cnt_headword_hira,cnt_headword,spell_pron_type,has_freq,fused_freq,freq_sum_headword,hira_freq_per,hira_order,pron_index
6733,の,の,57221425.0,no,686025.0,1,1,0,1,43219589.0,43219589.0,1.0,10,1
6391,に,に,35286930.0,NaN,0.0,1,1,0,1,23536382.0,23536382.0,1.0,10,1
9465,を,を,33707106.0,NaN,0.0,1,1,0,1,22482640.0,22482640.0,1.0,10,1
6865,は,は,31279365.0,NaN,0.0,1,1,0,1,20863336.0,20863336.0,1.0,10,1
2400,が,が,30705135.0,NaN,0.0,1,1,0,1,20480325.0,20480325.0,1.0,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,いい／よい,いい／よい,0.0,NaN,0.0,1,1,1,0,0.0,0.0,NaN,10,1
387,あふれる,あふれる,0.0,NaN,0.0,1,1,1,0,0.0,0.0,NaN,10,1
378,あの人,あのひと,0.0,NaN,0.0,1,1,1,0,0.0,0.0,NaN,10,1
333,あちこち,あちこち,0.0,NaN,0.0,1,1,1,0,0.0,0.0,NaN,10,1


In [27]:
# df_merged_raw 关联 df_merged_raw_word_agg的ID主键(减少一些列)
df_merged_raw_word_agg_simple = df_merged_raw_word_agg[
    ["headword", "word_id", "word_fused_freq"]
].copy()


df_merged_raw2 = df_merged_raw.reset_index()

df_hira = pd.merge(
    df_merged_raw2, df_merged_raw_word_agg_simple, on="headword", how="left"
)

df_hira["word_id"] = df_hira["word_id"].astype(str)

df_hira["pron_id"] = df_hira["word_id"] + df_hira["hira_order"]
df_hira.sort_values(
    by=["word_fused_freq", "headword", "hira_order"],
    ascending=[False, False, True],
    inplace=True,
)
df_hira[0:3]
to_csv_sig(df_hira, "D:/Dropbox/06.wanjuan/99.tmp/result.csv", True)
l("df_hira")

----------------------------------------df_hira----------------------------------------


In [28]:
# 关联发音文件 (headword关联)
# df_merged_raw.reset_index(inplace=True)

df_hira["word_voice"] = df_hira["headword"].map(
    df_ff_5.set_index("headword_voice")["mp3_path"]
)
df_hira["hira_voice"] = df_hira["hira"].map(
    df_ff_5.set_index("headword_voice")["mp3_path"]
)

df_hira["voice"] = df_hira["hira_voice"].fillna(df_hira["word_voice"])
katsu = cutlet.Cutlet()
df_hira["roman"] = df_hira["hira"].apply(katsu.romaji)


order = [
    "pron_id",
    "word_id",
    "headword",
    "hira",
    "hira_freq_per",
    "roman",
    "pron_index",
    "word_fused_freq",
    "fused_freq",
    "voice",
    "spoken_freq",
    "written_freq",
    "cnt_headword",
    "spell_pron_type",
    "has_freq",
]

df_hira = df_hira[order]
df_hira_new = df_hira.sort_values(
    by=["word_fused_freq", "fused_freq"], ascending=[False, False]
)

df_hira_new = df_hira_new.rename(
    columns={
        "headword": "spell",
        "hira": "pron",
        "hira_freq_per": "pron_percent",
    }
)


to_csv_sig(df_hira_new, w_raw_merged_freq_sumed_csv)
p("df_hira--ok")
df_hira_new[:3]

df_hira--ok


,pron_id,word_id,spell,pron,pron_percent,roman,pron_index,word_fused_freq,fused_freq,voice,spoken_freq,written_freq,cnt_headword,spell_pron_type,has_freq
0,10000010,100000,の,の,1.0,No,1,43219589.0,43219589.0,skent/の.mp3|poyotan/の.mp3|otiose/の.mp3|akiconf...,57221425.0,686025.0,1,0,1
1,10000110,100001,に,に,1.0,Ni,1,23536382.0,23536382.0,akiconfig/に.mp3|otiose/に.mp3|strawberrybrown/に...,35286930.0,0.0,1,0,1
2,10000210,100002,を,を,1.0,Wo,1,22482640.0,22482640.0,skent/を.mp3|kaoring/を.mp3|strawberrybrown/を.mp...,33707106.0,0.0,1,0,1


In [29]:
df_hira_new.tail()

,pron_id,word_id,spell,pron,pron_percent,roman,pron_index,word_fused_freq,fused_freq,voice,spoken_freq,written_freq,cnt_headword,spell_pron_type,has_freq
104276,20230710,202307,いい／よい,いい／よい,NaN,Ii/yoi,1,0.0,0.0,NaN,0.0,0.0,1,1,0
104277,20233610,202336,あふれる,あふれる,NaN,Afureru,1,0.0,0.0,tanukitosanpo1/あふれる.mp3|Mikage/あふれる.mp3,0.0,0.0,1,1,0
104278,20233510,202335,あの人,あのひと,NaN,Ano hito,1,0.0,0.0,kaoring/あのひと.mp3,0.0,0.0,1,1,0
104279,20236810,202368,あちこち,あちこち,NaN,Achikochi,1,0.0,0.0,usako_usagiclub/あちこち.mp3|strawberrybrown/あちこち....,0.0,0.0,1,1,0
104280,20244810,202448,あいさつ,あいさつ,NaN,Aisatsu,1,0.0,0.0,Mikage/あいさつ.mp3|poyotan/あいさつ.mp3,0.0,0.0,1,1,0
